# RNNs para Martín Fierro

El objetivo de los ejercicios en este tutorial es mostrar el impacto de algunas decisiones de diseño en la implementación de las redes neuronales, particularmente las recurrentes. Como ejemplo veremos una implementación de la red RNN para generación de lenguaje basada en caracteres de [Karpathy](http://karpathy.github.io/2015/05/21/rnn-effectiveness/). Para entrenarla utilizaremos un fragmento del Martín Fierro que pueden descargar [aquí](https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/martin_fierro.txt). Para un entrenamiento más complejo, pueden utilizar las obras completas de borges, disponibles en [este link](https://drive.google.com/file/d/0B4remi0ZCiqbUFpTS19pSmVFYkU/view?usp=sharing).


In [1]:
from __future__ import absolute_import, print_function, unicode_literals

import numpy as np
import random
import re
import sys
import unicodedata

Primero leeremos el dataset del archivo de texto y lo preprocesaremos para disminuir la viariación de caracteres. Normalizaremos el formato unicos, elminaremos espacios y transformaremos todo a minúsculas.

In [5]:
%%bash

wget https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/martin_fierro.txt

--2018-09-21 15:37:47--  https://cs.famaf.unc.edu.ar/~mteruel/datasets/diplodatos/martin_fierro.txt
Resolving cs.famaf.unc.edu.ar (cs.famaf.unc.edu.ar)... 200.16.17.55
Connecting to cs.famaf.unc.edu.ar (cs.famaf.unc.edu.ar)|200.16.17.55|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 35910 (35K) [text/plain]
Saving to: 'martin_fierro.txt.2'

     0K .......... .......... .......... .....                100% 43.6M=0.001s

2018-09-21 15:37:47 (43.6 MB/s) - 'martin_fierro.txt.2' saved [35910/35910]



In [6]:
with open('./martin_fierro.txt', 'r') as finput:
    text = unicodedata.normalize('NFC', finput.read()).lower()
    text = re.sub('\s+', ' ', text).strip()

print('Corpus length: %d' % len(text))

Corpus length: 33858


Luego, contaremos la cantidad de caracteres únicos presentes en el texto, y le asignaremos a cada uno un índice único y secuencial. Este índice será utilizado luego para crear las representaciones one-hot encoding de los caracteres.

In [7]:
chars = sorted(list(set(text)))

print('Total chars: %d' % len(chars))

char_indices = dict((c, i) for i, c in enumerate(chars))
indices_char = dict((i, c) for i, c in enumerate(chars))

Total chars: 54


## Parte 1: Esqueleto de la red neuronal

Lo primero que debemos pensar es cómo será la arquitectura de nuestra red para resolver la tarea deseada. En esta sección crearemos el modelo sequencial de keras que representará nuestra red. En los pasos siguientes, implementaremos las transformaciones del corpus, por lo que en este paso pueden asumir cualquier formato en los datos de entrada.

Para poder implementar el modelo debemos responder las siguientes preguntas:
  - ¿Es una red one-to-one, one-to-many, many-to-one o many-to-many?
  - ¿Cuál es el formato de entrada y de salida de la red? ¿Cuál es el tamaño de las matrices (tensores) de entrada y de salida?
  - Luego de que la entrada pasa por la capa recurrente, ¿qué tamaño tiene el tensor?
  - ¿Cómo se conecta la salida de la capa recurrente con la capa densa que realiza la clasificación?
  - ¿Cuál es el loss apropiado para este problema?

Las funciones de Keras que tendrán que utilizar son:
  - keras.layers.LSTM
  - keras.layers.TimeDistributed
  - keras.layers.Dense

In [8]:
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import LSTM, TimeDistributed
from keras.optimizers import RMSprop

import keras.backend as K

Using TensorFlow backend.


In [9]:
K.clear_session()

# build the model: a single LSTM
model = Sequential()
hidden_layer_size = 128
maxlen = 40
model.add(LSTM(hidden_layer_size, input_shape=(maxlen, len(chars)), return_sequences=True))
# The output of the network at this point has shape (batch_size, maxlen, hidden_layer_size)
# We need to convert it into something of shape (batch_size, maxlen, len(chars))
# by applying THE SAME dense layer to all the times in the sequence.
model.add(TimeDistributed(Dense(len(chars), activation='softmax')))

model.compile(loss='categorical_crossentropy', optimizer='adam')
model.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm_1 (LSTM)                (None, 40, 128)           93696     
_________________________________________________________________
time_distributed_1 (TimeDist (None, 40, 54)            6966      
Total params: 100,662
Trainable params: 100,662
Non-trainable params: 0
_________________________________________________________________


## Parte 2: Transformación del input

Una vez que definimos la arquitectura de la red, sabemos con exactitud cuál es el input que necesitamos utilizar. En esta sección transformaremos el texto que leimos del archivo en ejemplos de entrenamiento para nuestra red. El resultado será una matrix que representa las secuencias de caracteres y una matriz que representa las etiquetas correspondientes.

  - ¿Cómo debemos representar cada ejemplo?
  - ¿Cómo debemos representar cada etiqueta?

In [10]:
# cut the text in sequences of maxlen characters
sentences = []
next_chars = []

for i in range(0, len(text) - maxlen - 1, maxlen):
    sentences.append(text[i: i + maxlen])
    next_chars.append(text[i + 1: i + maxlen + 1])

print('NB sequences:', len(sentences))

NB sequences: 846


In [11]:
X = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
y = np.zeros((len(sentences), maxlen, len(chars)), dtype=np.bool)
for i, sentence in enumerate(sentences):
    for t, char in enumerate(sentence):
        X[i, t, char_indices[char]] = 1
        y[i, t, char_indices[next_chars[i][t]]] = 1

## Parte 3: Entrenamiento de la red

En esta sección entrenaremos nuestra red llamando al método ´fit´ de keras. Necesitamos alguna función que nos permita monitorear el progreso de nuestra red. Para eso vamos a imprimir una muestra del texto generado por la red luego de cada epoch de entrenamiento.

Para ello, utilizaremos dos funciones que toman una porción de texto aleatorio y generan nuevos caracteres con el modelo dado. 

    - ¿Cómo podemos interpretar la salida de la red? ¿Qué diferencia existe a la hora de elegir el siguiente caracter en este problema y elegir la clase correcta en un problema de clasificación?
    - ¿Qué hacen estas funciones? ¿Para qué se utiliza la variable diversity?

In [12]:
def sample(preds, temperature=1.0):
    # helper function to sample an index from a probability array
    preds = np.asarray(preds).astype('float64')
    preds = np.log(preds) / temperature
    exp_preds = np.exp(preds)
    preds = exp_preds / np.sum(exp_preds)
    probas = np.random.multinomial(1, preds, 1)
    return np.argmax(probas)

In [13]:
def print_samples(model, sample_size=400):
    start_index = random.randint(0, len(text) - maxlen - 1)

    for diversity in [0.2, 0.5, 1.0, 1.2]:
        print()
        print('----- diversity:', diversity)

        sentence = text[start_index: start_index + maxlen]
        print('----- Generating with seed: "' + sentence + '"')
        sys.stdout.write(sentence)

        # Printing the sample
        for i in range(sample_size):
            x = np.zeros((1, maxlen, len(chars)))
            # Build the one-hot encoding for the sentence
            for t, char in enumerate(sentence):
                x[0, t, char_indices[char]] = 1.

            preds = model.predict(x, verbose=0)[0][-1]
            next_index = sample(preds, diversity)
            next_char = indices_char[next_index]

            sentence = sentence[1:] + next_char

            sys.stdout.write(next_char)
            sys.stdout.flush()
        print()

In [14]:
last_loss = -1.

for iteration in range(1, 13):
    print()
    print('-' * 50)
    print('Iteration', iteration)
    history = model.fit(X, y,
                        batch_size=256,
                        epochs=1)

    if iteration % 3 == 0:
        print_samples(model)
    if last_loss >= 0 and last_loss - history.history['loss'][0] < 0.001:
        break
    
    last_loss = history.history['loss'][0]


--------------------------------------------------
Iteration 1
Epoch 1/1
846/846 [==============================] - 1s 1ms/step - loss: 3.9892

--------------------------------------------------
Iteration 2
Epoch 1/1
846/846 [==============================] - 0s 464us/step - loss: 3.9433

--------------------------------------------------
Iteration 3
Epoch 1/1
846/846 [==============================] - 0s 468us/step - loss: 3.8529

----- diversity: 0.2
----- Generating with seed: " 215 sólo vía sino hacienda y cielo. cua"
 215 sólo vía sino hacienda y cielo. cua        r  u  i                       a     d                                                       a                                                                                                            l                   r        a                         a                                     a      a                                         a                    d                           

----- diversity: 0.5
----- Ge

o: -vas a saber »si es solo o acompañao»         a        a i      a a                e e                    e      a                  a           e       a   n   e     o      a   a  a e    r   a  e  e     o a     a e        a         ea       e  a     e  e                             e                e                                 e                         e                    a    e  e  na                      a   a       ea e     

----- diversity: 0.5
----- Generating with seed: "o: -vas a saber »si es solo o acompañao»"
o: -vas a saber »si es solo o acompañao»  o na   s e a oea  sano   u c d amedtien  o a  a   a c  aa lu    cn le oaer  r maah a    i  noee  e   r   oea    s ra a re   s oeo     sseee  u     l ae eamnee o n o eoea u   e ec     or    o   e lelto an eg    eee eaouo aa  eeeu aeia   aatha c ieere  sau  e  a   to aao     ol oaioad aiyor ao   ar sm pn usa   aa  us a lnb sueas ean le e aueeeaas a enee s ee  e a    oenae  e n oappn saea  na ao eu

----- diversity: 1.0
---

## Ejercicios extras

Una vez que hemos implementado la arquitectura básica de la red, podemos comenzar a experimentar con distintas modificaciones para lograr mejores resultados. Algunas tareas posibles son:

 - Agregar más capas recurrentes
 - Probar otras celdas recurrentes
 - Probar otros largos de secuencias máximas
 - Agregar capas de regularización y/o dropout
 - Agregar métricas de performance como perplexity y word error rate